#### Notes
#### Returns
#### Amazon

<br>

### Future notes
##### remove/clean up columns (like controllables)


In [12]:
import polars as pl
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
import time
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

In [13]:
df = pl.read_csv(r"C:\Users\Denne\Documents\VSCodeProjects\FinancialStreamline2\BankStreamline\Data\BankDataProd.csv", ignore_errors=True)


df = df.with_columns(
    pl.col("date").str.strptime(pl.Date, "%Y-%m-%d", strict=False)
)


# End date
days_backward = time.localtime().tm_mday
now = datetime.now()
EndOfLastMonth = now - timedelta(days=days_backward)
# Start date
LastYear = EndOfLastMonth - relativedelta(years=1)
StartDate = LastYear + relativedelta(months=1)
StartDate = StartDate.replace(day=1).date()
last_month_int = EndOfLastMonth.month


month_order = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", 
               "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

df = df.sort("month")


df = df.filter(pl.col("date") >= pl.date(StartDate.year, StartDate.month, StartDate.day)) \
        .filter(pl.col("date") <= pl.date(EndOfLastMonth.year, EndOfLastMonth.month, EndOfLastMonth.day))

df.limit(3)

date,category,description,cost,monthName,month,day,year,weekDay,cardType,quarter
date,str,str,f64,str,i64,i64,i64,str,str,i64
2026-01-31,"""wmt""","""POS Withdrawal WALMART.COM 800…",74.5,"""Jan""",1,31,2026,"""Sat""","""Debit""",1
2026-01-30,"""wmt""","""Card purchase WALMART.COM 800…",112.49,"""Jan""",1,30,2026,"""Fri""","""Debit""",1
2026-01-30,"""wmt""","""POS Withdrawal WAL-MART #0530 …",12.5,"""Jan""",1,30,2026,"""Fri""","""Debit""",1


### OG Graph Remake

In [15]:
# tempDF = df.group_by(["monthName", 'month']).agg(pl.sum("cost")) 

fig = px.bar(df.to_pandas(), x="monthName", y="cost", color="category", hover_data=['description', 'cardType', 'day'],
             title="Total Expense by Month",
             category_orders={"monthName": month_order},
             template='plotly_dark',
             labels={'monthName':''},
             text_auto=True,
             height=800,
             width=1600)

fig.show()
# fig.show()

### Smooth Graph Remake

In [16]:
SMOOTH = 1

if SMOOTH == 0:
    tempDF = df.group_by(["monthName", 'month', 'category']).agg(pl.sum("cost")) 
    fig = px.bar(tempDF.to_pandas(), x="monthName", y="cost", color="category",
             title="Total Expense by Month",
             category_orders={"monthName": month_order},
             template='plotly_dark',
             labels={'monthName':''},
             text_auto=True,
             height=800,
             width=1600)

else:
    tempDF = df.group_by(["monthName", 'month']).agg(pl.sum("cost")) 
    fig = px.bar(tempDF.to_pandas(), x="monthName", y="cost",
             title="Total Expense by Month",
             category_orders={"monthName": month_order},
             template='plotly_dark',
             labels={'monthName':''},
             text_auto=True,
             height=800,
             width=1600)



fig.show()

### Only October Numbers (wip)

In [17]:
tempDF = df.filter(pl.col('month') == last_month_int) \
            .group_by(['category']).agg(pl.sum("cost")) 

tempDF = tempDF.sort(by='cost', descending=True)

fig = px.bar(tempDF.to_pandas(), x="category", y="cost", color="category",
             title="Last Month Expenses by Category",
            #  category_orders={"monthName": month_order},
             template='plotly_dark',
             labels={'monthName':''},
             text_auto=True,
             height=800,
             width=1600)

fig.show()

### Last month against LTM average 
##### Data Setup

In [18]:
avgs = df.group_by(['category']).agg(pl.sum("cost").alias('LTM_avg')) 

avgs = avgs.with_columns(LTM_avg = pl.col('LTM_avg') / 12)

last_month_df = df.filter(month = last_month_int).select('category', 'monthName', 'cost').group_by(['category', 'monthName']).agg(pl.sum("cost").alias('last_month'))

last_month_df
avgs


joined_df = last_month_df.join(avgs, on='category', how='inner')  \
            .with_columns(difference = pl.col('LTM_avg') - pl.col('last_month'))

joined_df




category,monthName,last_month,LTM_avg,difference
str,str,f64,f64,f64
"""subscriptions""","""Jan""",20.76,42.190833,21.430833
"""gas""","""Jan""",65.21,111.06,45.85
"""food""","""Jan""",115.99,117.025,1.035
"""power""","""Jan""",13.8,116.449167,102.649167
"""water""","""Jan""",79.99,36.975833,-43.014167
…,…,…,…,…
"""tithing""","""Jan""",533.62,549.788333,16.168333
"""shopping""","""Jan""",60.31,52.114167,-8.195833
"""internet""","""Jan""",79.0,105.546667,26.546667


### Last month against LTM average 
##### Graph

In [19]:
list(df['category'].unique())

['progressive',
 'wmt',
 'tithing',
 'fun',
 'food',
 'water',
 'amazon',
 'rent',
 'misc',
 'gas',
 'power',
 'car',
 'home improvement',
 'subscriptions',
 'natural gas',
 'shopping',
 'internet']

In [20]:

fig = go.Figure()

fig.update_layout(width=2000, height=1200, template="plotly_dark")


def create_guage(category, x1, x2, y1, y2):
    tf = joined_df.filter(pl.col("category").str.to_lowercase() == category.lower())

    if tf.is_empty():
        print(f"⚠️ No data for category: {category}")
        return

    lm = tf.select(pl.col("last_month").max()).item()
    ltm = tf.select(pl.col("LTM_avg").max()).item()

    if lm is None or ltm is None:
        print(f"⚠️ Missing values for {category}")
        return

    bar_color = 'red' if lm > ltm else 'green'

    fig.add_trace(go.Indicator(
        mode="number+gauge+delta", value=lm,
        delta={'reference': ltm,
               'increasing': {'color': "red"},
               'decreasing': {'color': "green"}},
        domain={'x': [x1, x2], 'y': [y1, y2]},
        title={'text': category},
        gauge={'steps': [{'range': [0, ltm], 'color': 'gray'}],
               'bar': {'color': bar_color}}
    ))


# Row 0 (top)
create_guage('car',          0.00, 0.16, 0.80, 1.00)
create_guage('internet',     0.20, 0.36, 0.80, 1.00)
create_guage('tithing',      0.40, 0.56, 0.80, 1.00)
create_guage('rent',         0.60, 0.76, 0.80, 1.00)
create_guage('wmt',          0.80, 0.96, 0.80, 1.00)

# Row 1
create_guage('progressive',  0.00, 0.16, 0.55, 0.75)
create_guage('shopping',     0.20, 0.36, 0.55, 0.75)
create_guage('natural gas',  0.40, 0.56, 0.55, 0.75)
create_guage('amazon',       0.60, 0.76, 0.55, 0.75)
create_guage('subscriptions',0.80, 0.96, 0.55, 0.75)

# Row 2
create_guage('fun',          0.00, 0.16, 0.30, 0.50)
create_guage('home improvement', 0.20, 0.36, 0.30, 0.50)
create_guage('power',        0.40, 0.56, 0.30, 0.50)
create_guage('misc',         0.60, 0.76, 0.30, 0.50)
create_guage('food',         0.80, 0.96, 0.30, 0.50)
def create_guage(category, x1, x2, y1, y2):
    tf = joined_df.filter(pl.col("category").str.to_lowercase() == category.lower())

    if tf.is_empty():
        print(f"⚠️ No data for category: {category}")
        return

    lm = tf.select(pl.col("last_month").max()).item()
    ltm = tf.select(pl.col("LTM_avg").max()).item()

    if lm is None or ltm is None:
        print(f"⚠️ Missing values for {category}")
        return

    bar_color = 'red' if lm > ltm else 'green'

    fig.add_trace(go.Indicator(
        mode="number+gauge+delta", value=lm,
        delta={'reference': ltm,
               'increasing': {'color': "red"},
               'decreasing': {'color': "green"}},
        domain={'x': [x1, x2], 'y': [y1, y2]},
        title={'text': category},
        gauge={'steps': [{'range': [0, ltm], 'color': 'gray'}],
               'bar': {'color': bar_color}}
    ))
# Row 3 (bottom)
create_guage('gas',          0.00, 0.16, 0.05, 0.25)
create_guage('water',        0.20, 0.36, 0.05, 0.25)



fig.show()

⚠️ No data for category: car
⚠️ No data for category: fun


## Junk for reference

In [21]:
fig = go.Figure()

fig.add_trace(go.Indicator(
    mode = "number+gauge+delta", value = 180,
    delta = {'reference': 150},
    # domain = {'x': [0.25, 1], 'y': [0.08, 0.25]},
    title = {'text': "Revenue"},
    gauge = {
        'steps': [{'range': [0, 150], 'color': "gray"}],
        'bar': {'color': "black"}}))

fig.update_layout(width=1200)